# Comparison between  neural networks with different hyperparameter combinations

Load all necessary packages

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
import joblib
import matplotlib.pyplot as plt 
from sklearn.model_selection import KFold

plt.style.use('bmh')

import matplotlib
matplotlib.rcParams.update({'font.size': 16})

from tqdm import tqdm
%load_ext autoreload
%autoreload 2
%aimport IS_misc_functions
from IS_misc_functions import *
from misc_functions import *
from actuarial_training import *

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler,normalize
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import json

# 1.) Actuarial Approach

Define the model, load the data, and pretrained approximated GA

In [2]:

def build_model(x,nr_neurons = 256,n_layers =4, Dropout_rate = 0.2):  
    
    model = Sequential()
    for _ in range(n_layers):
        model.add(layers.Dense(nr_neurons, input_shape=(401,), activation='relu'))
        model.add(layers.Dropout(Dropout_rate))
    model.add(layers.Dense(1, activation='linear'))

    opt1 = tf.keras.optimizers.Adam(learning_rate = 0.0005, beta_1=0.9, beta_2=0.999)
    model.compile(loss=tf.keras.losses.MeanSquaredError(), #tf.keras.losses.MeanAbsolutePercentageError(), #mse',
                    optimizer=opt1,
                    metrics=[tf.keras.losses.MeanAbsoluteError(), tf.keras.losses.MeanSquaredError()])
    scaler = StandardScaler().fit(x)
    return model, scaler
    
# # Read Data
def read_data(xpath,ypath):
    df_x = pd.read_csv(xpath, header=None)
    df_x = df_x.dropna()
    df_x = df_x.iloc[:,1:]

    df_y = pd.read_csv(ypath, header=None)
    df_y = df_y.dropna()
    df_y = df_y.iloc[:,1:]
    
    return df_x,df_y
x,y = read_data("csv/X_1.csv","csv/Y_1.csv")
for i in range(2,11):
    x_new,y_new = read_data("csv/X_"+str(i)+".csv","csv/Y_"+str(i)+".csv")
    x = pd.concat([x,x_new])
    y = pd.concat([y,y_new])
x_train = x.reset_index(drop = True)
y_train = y.reset_index(drop = True)

x_test, y_test = read_data("csv/X_test.csv","csv/Y_test.csv")


x_train_approx = pd.read_csv("csv/X_Train_act_approx.csv", header=None).iloc[1:,1].reset_index(drop = True)
x_test_approx = pd.read_csv("csv/X_Test_act_approx.csv", header=None).iloc[1:,1]

x_train = pd.concat([x_train,x_train_approx],axis = 1)
x_test = pd.concat([x_test,x_test_approx],axis = 1)

Now train different models, and compare the results

In [3]:
# Parameters
EPOCHS = 1000
BATCH_SIZE = 32

neurons_set = [128, 512, 1024]
Dropout_rate_set = [0.1, 0.2]
n_layers_set = [2, 5,8]
n_folds = 5

# Initialize results
results = []

# Set up k-fold cross-validation
kf = KFold(n_splits=n_folds)

# Loop through different hyperparameter combinations
for nr_neurons in neurons_set:
    for Dropout_rate in Dropout_rate_set:
        for n_layers in n_layers_set:
            fold_errors = []
            
            # Cross-validation loop
            for train_index, val_index in kf.split(x_train):
                # Split data into training and validation sets for this fold
                x_train_fold, x_val_fold = x_train.iloc[train_index], x_train.iloc[val_index]
                y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]
                
                # Build and scale model
                model, scaler = build_model(x_train_fold, nr_neurons, n_layers, Dropout_rate)
                x_train_scaled = scaler.fit_transform(x_train_fold)
                x_val_scaled = scaler.transform(x_val_fold)
                x_test_scaled = scaler.transform(x_test)
                
                early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=200)
                
                # Fit the model
                model.fit(x_train_scaled, y_train_fold, epochs=EPOCHS, batch_size=BATCH_SIZE,
                          validation_data=(x_val_scaled, y_val_fold), verbose=0, callbacks=[early_stop])
                
                # Evaluate on the test set for this fold
                GA_NN_List_999 = []
                GA_List_999 = []
                for i in range(len(x_test)):
                    GA_NN_List_999.append(model.predict(np.reshape(x_test_scaled[i, :], (1, 401))))
                    GA_List_999.append(y_test.iloc[i, :])
                
                # Calculate the error for this fold
                diff_MC_NN_999 = np.array([np.abs(i[0] - j) for i, j in zip(GA_NN_List_999, GA_List_999)])
                fold_errors.append(np.mean(diff_MC_NN_999))
            
            # Calculate the mean error across all folds
            mean_error = np.mean(fold_errors)
            
            # Store the result
            comparison_dict = {"Layers": n_layers, "Neurons": nr_neurons, 
                               "Dropout Rate": Dropout_rate, "Mean Error": mean_error}
            results.append(comparison_dict)


Print the results

In [4]:
pd.DataFrame(results)

,Layers,Neurons,Dropout Rate,Mean Error
0,2,128,0.1,0.005423
1,5,128,0.1,0.005955
2,8,128,0.1,0.008841
3,2,128,0.2,0.005481
4,5,128,0.2,0.007196
5,8,128,0.2,0.013086
6,2,512,0.1,0.005508
7,5,512,0.1,0.005869
8,8,512,0.1,0.006315
9,2,512,0.2,0.005733


# 2.) MtM Approach

Define the model, load the data, and pretrained approximated GA

In [2]:
def build_model(x,nr_neurons = 256,n_layers =4, Dropout_rate = 0.2):  
    #Input Layer
    full_input = keras.Input(shape=(601,),name = "full_input")
    
#     # Reduction to the case wher ELGD,c and D is the same for all obligors
    small_input_1 = tf.reshape(full_input[:,0],(-1,1))
    small_input_2 = full_input[:,100:300]
    small_input_3 = tf.reshape(full_input[:,300],(-1,1))
    small_input_4 = full_input[:,400:500]
    small_input_5 = tf.reshape(full_input[:,500],(-1,1))
    small_input_6 = tf.reshape(full_input[:,600],(-1,1))
    # concatenate
    v = layers.concatenate([small_input_1,small_input_2,small_input_3,
                            small_input_4,small_input_5,small_input_6])
    #Batch Normalization applied to the input
    v = layers.BatchNormalization()(v)

    # Create the NN
    v = layers.Dense(nr_neurons,activation = "relu")(v) #,kernel_regularizer=tf.keras.regularizers.L1(0.01))(v)
    # Create deep layers
    for i in range(n_layers):
        v = layers.Dense(nr_neurons,activation =  "relu")(v) #,kernel_regularizer=tf.keras.regularizers.L1(0.01))(v)
        v = layers.Dropout(Dropout_rate)(v)
    # Output Layers
    value_out = layers.Dense(1)(v)
    model = keras.Model(inputs=[full_input],outputs = [value_out])
    return model
# # Read Data
def read_data(xpath,ypath):
    df_x = pd.read_csv(xpath, header=None)
    df_x = df_x.dropna()
    df_x = df_x.iloc[:,1:]

    df_y = pd.read_csv(ypath, header=None)
    df_y = df_y.dropna()
    df_y = df_y.iloc[:,1:]
    
    return df_x,df_y
x,y = read_data("csv/X_mtm_1.csv","csv/Y_mtm_1.csv")
for i in range(2,15):
    x_new,y_new = read_data("csv/X_mtm_"+str(i)+".csv","csv/Y_mtm_"+str(i)+".csv")
    x = pd.concat([x,x_new])
    y = pd.concat([y,y_new])
x_train = x.reset_index(drop = True)
y_train = y.reset_index(drop = True)

x_test, y_test = read_data("csv/X_mtm_test.csv","csv/Y_mtm_test.csv")

x_train_approx = pd.read_csv("csv/X_Train_MtM_approx.csv", header=None).iloc[1:,1].reset_index(drop = True)
x_test_approx = pd.read_csv("csv/X_Test_MtM_approx.csv", header=None).iloc[1:,1]

x_train = pd.concat([x_train,x_train_approx],axis = 1)
x_test = pd.concat([x_test,x_test_approx],axis = 1)

Train different models

In [ ]:
# Parameters
EPOCHS = 1000
BATCH_SIZE = 32

neurons_set = [128, 512, 1024]
Dropout_rate_set = [0.1, 0.2]
n_layers_set = [2, 5,8]
n_folds = 5

# Initialize results
results = []

# Set up k-fold cross-validation
kf = KFold(n_splits=n_folds)

# Loop through different hyperparameter combinations
for nr_neurons in neurons_set:
    for Dropout_rate in Dropout_rate_set:
        for n_layers in n_layers_set:
            fold_errors = []
            
            # Cross-validation loop
            for train_index, val_index in kf.split(x_train):
                # Split data into training and validation sets for this fold
                x_train_fold, x_val_fold = x_train.iloc[train_index], x_train.iloc[val_index]
                y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]
                
                # Build and compile model
                model = build_model(x_train_fold, nr_neurons, n_layers, Dropout_rate)
                opt1 = tf.keras.optimizers.Adam(learning_rate=0.00005, beta_1=0.9, beta_2=0.999)
                model.compile(loss=["mse", tf.keras.losses.MeanAbsolutePercentageError()],
                              loss_weights=[1, 0],  # Only using MSE loss
                              optimizer=opt1,
                              metrics=['mae', 'mse'])
                
                # Scale the data for the fold
                scaler = StandardScaler().fit(x_train_fold)
                x_train_scaled = scaler.transform(x_train_fold)
                x_val_scaled = scaler.transform(x_val_fold)
                x_test_scaled = scaler.transform(x_test)
                
                early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=200)
                
                # Fit the model for this fold
                model.fit(x_train_scaled, y_train_fold, epochs=EPOCHS, batch_size=BATCH_SIZE,
                          validation_data=(x_val_scaled, y_val_fold), verbose=0, callbacks=[early_stop])
                
                # Evaluate on the test set for this fold
                GA_NN_List_999 = []
                GA_List_999 = []
                for i in range(len(x_test)):
                    GA_NN_List_999.append(model.predict(np.reshape(x_test_scaled[i, :], (1, 601))))
                    GA_List_999.append(y_test.iloc[i, :])
                
                # Calculate the error for this fold
                diff_MC_NN_999 = np.array([np.abs(i[0] - j) for i, j in zip(GA_NN_List_999, GA_List_999)])
                fold_errors.append(np.mean(diff_MC_NN_999))
            
            # Calculate the mean error across all folds
            mean_error = np.mean(fold_errors)
            
            # Store the result
            comparison_dict = {"Layers": n_layers, "Neurons": nr_neurons, 
                               "Dropout Rate": Dropout_rate, "Mean Error": mean_error}
            results.append(comparison_dict)


In [ ]:
# Parameters
EPOCHS = 1000
BATCH_SIZE = 32

neurons_set = [512, 1024]
Dropout_rate_set = [0.1, 0.2]
n_layers_set = [2, 5,8]
n_folds = 5

# Initialize results
# results = []

# Set up k-fold cross-validation
kf = KFold(n_splits=n_folds)

# Loop through different hyperparameter combinations
for nr_neurons in neurons_set:
    for Dropout_rate in Dropout_rate_set:
        for n_layers in n_layers_set:
            fold_errors = []
            
            # Cross-validation loop
            for train_index, val_index in kf.split(x_train):
                # Split data into training and validation sets for this fold
                x_train_fold, x_val_fold = x_train.iloc[train_index], x_train.iloc[val_index]
                y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]
                
                # Build and compile model
                model = build_model(x_train_fold, nr_neurons, n_layers, Dropout_rate)
                opt1 = tf.keras.optimizers.Adam(learning_rate=0.00005, beta_1=0.9, beta_2=0.999)
                model.compile(loss=["mse", tf.keras.losses.MeanAbsolutePercentageError()],
                              loss_weights=[1, 0],  # Only using MSE loss
                              optimizer=opt1,
                              metrics=['mae', 'mse'])
                
                # Scale the data for the fold
                scaler = StandardScaler().fit(x_train_fold)
                x_train_scaled = scaler.transform(x_train_fold)
                x_val_scaled = scaler.transform(x_val_fold)
                x_test_scaled = scaler.transform(x_test)
                
                early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=200)
                
                # Fit the model for this fold
                model.fit(x_train_scaled, y_train_fold, epochs=EPOCHS, batch_size=BATCH_SIZE,
                          validation_data=(x_val_scaled, y_val_fold), verbose=0, callbacks=[early_stop])
                
                # Evaluate on the test set for this fold
                GA_NN_List_999 = []
                GA_List_999 = []
                for i in range(len(x_test)):
                    GA_NN_List_999.append(model.predict(np.reshape(x_test_scaled[i, :], (1, 601))))
                    GA_List_999.append(y_test.iloc[i, :])
                
                # Calculate the error for this fold
                diff_MC_NN_999 = np.array([np.abs(i[0] - j) for i, j in zip(GA_NN_List_999, GA_List_999)])
                fold_errors.append(np.mean(diff_MC_NN_999))
            
            # Calculate the mean error across all folds
            mean_error = np.mean(fold_errors)
            
            # Store the result
            comparison_dict = {"Layers": n_layers, "Neurons": nr_neurons, 
                               "Dropout Rate": Dropout_rate, "Mean Error": mean_error}
            results.append(comparison_dict)


In [ ]:
pd.DataFrame(results)